In [ ]:
import torch
import torchvision
import torchvision.datasets as dset
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.nn as nn
from torch import optim
from torch.autograd import Variable

In [ ]:
learning_rate = 1e-3
batch_size = 64
epoches = 50

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
trans_img = transforms.ToTensor()
train = dset.MNIST(root='../LeNet', train=True, transform=trans_img)
test = dset.MNIST(root='../LeNet', train=False, transform=trans_img)

In [ ]:
train

In [ ]:
test

In [ ]:
trainLoader = DataLoader(train, batch_size=batch_size, shuffle=True, num_workers=4)
teseLoader = DataLoader(test, batch_size=batch_size, shuffle=False, num_workers=4)

In [ ]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Sequential(
        nn.Conv2d(1, 6, 5, stride=1, padding=2),
        nn.ReLU(),
        nn.MaxPool2d(2, 2)
        )
        self.conv2 = nn.Sequential(
        nn.Conv2d(6, 16, 5, stride=1, padding=0),
        nn.ReLU(),
        nn.MaxPool2d(2, 2)
        )
        self.fc1 = nn.Sequential(
        nn.Linear(16*5*5, 120),
        nn.ReLU()
        )
        self.fc2 = nn.Sequential(
        nn.Linear(120, 84),
        nn.ReLU()
        )
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size()[0], -1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

In [ ]:
lenet = LeNet()
if torch.cuda.is_available():
    lenet.cuda()

In [ ]:
criterian = nn.CrossEntropyLoss(size_average=False)
optimizer = optim.SGD(lenet.parameters(), lr=learning_rate)

In [ ]:
for epoch in range(epoches):
    sum_loss = 0.0
    sum_acc = 0
    
    for (img, label) in trainLoader:
        img = img.to(device)
        label = label.to(device)
        
        optimizer.zero_grad()
        output = lenet(img)
        loss = criterian(output, label)
        # backward
        loss.backward()
        optimizer.step()
        
        sum_loss += loss.item()
        _, predict = torch.max(output, 1)
        correct_num = (predict==label).sum()
        sum_acc += correct_num.item()
    
    sum_loss /= len(train)
    sum_acc /= len(train)
    print("[%d/%d] Loss: %.5f, Acc: %.2f" %(i+1, epoches, sum_loss, 100*sum_acc))